En esta tarea, exploraremos count-min sketches y filtros Bloom. Utilizaremos dos archivos de texto, `great-gatsby-fitzgerald.txt` y `war-and-peace-tolstoy.txt`, para cargar el texto de dos novelas famosas, cortesía del proyecto Guttenberg.

Exploraremos dos tareas:

  - Contar la frecuencia de palabras de 5 o más caracteres en ambas novelas usando un count-min sketch
  - Usar un filtro Bloom para contar aproximadamente cuántas palabras de la novela *War and Peace* ya aparecen en *The Great Gatsby*.

#### **Paso 1: Crear una familia universal de funciones hash**

Utilizaremos una familia de funciones hash que comienza (a) generando un número primo aleatorio $p$ (usaremos la prueba de primalidad Miller-Rabin para este propósito); (b) generando números aleatorios **a** y **b** entre 2 y p-1.

La función hash $h_{a,b,p}(n) = (an + b) \mod p$.

Ten en cuenta que esta función produce valores entre 0 y $p-1$. También debemos asegurarnos de tomar el valor hash módulo $m$, donde $m$ es el tamaño de la tabla hash.

Para hashear cadenas, primero utilizaremos la función hash incorporada de Python y luego aplicaremos $h_{a,b,p}$ al resultado.

Como primer paso, generaremos un número primo aleatorio.

#### **(A) Generar números primos aleatorios**

In [ ]:
# Programa en Python 3 para la prueba de primalidad aleatoria Miller-Rabin
# Copiado de geeksforgeeks: https://www.geeksforgeeks.org/primality-test-set-3-miller-rabin/
import random 

# Función auxiliar para realizar la exponenciación modular.
# Retorna (x^y) % p
def power(x, y, p): 
	# Inicializar resultado
	res = 1; 
	
	# Actualizar x si es mayor o igual que p
	x = x % p; 
	while (y > 0): 
		# Si y es impar, multiplica x con el resultado
		if (y & 1): 
			res = (res * x) % p; 

		# Ahora y debe ser par
		y = y >> 1; # y = y/2 
		x = (x * x) % p; 
	
	return res; 

# Esta función se llama para todos los k ensayos.
# Retorna False si n es compuesto y True si n es probablemente primo.
# d es un número impar tal que d*2^r = n-1 para algún r >= 1
def miillerTest(d, n): 
	# Elegir un número aleatorio en [2, n-2]
	# Casos particulares aseguran que n > 4
	a = 2 + random.randint(1, n - 4); 

	# Calcular a^d % n
	x = power(a, d, n); 

	if (x == 1 or x == n - 1): 
		return True; 

	# Seguir elevando x al cuadrado hasta que:
	# (i) d no alcance n-1, o
	# (ii) (x^2) % n no sea 1, o
	# (iii) (x^2) % n no sea n-1
	while (d != n - 1): 
		x = (x * x) % n; 
		d *= 2; 

		if (x == 1): 
			return False; 
		if (x == n - 1): 
			return True; 

	# Retorna compuesto
	return False; 

# Retorna False si n es compuesto y True si n es probablemente primo.
# k es un parámetro que determina el nivel de precisión; un valor mayor de k indica mayor precisión.
def isPrime( n, k): 
	# Casos particulares
	if (n <= 1 or n == 4): 
		return False; 
	if (n <= 3): 
		return True; 

	# Encontrar d tal que n = 2^r * d + 1 para algún r >= 1
	d = n - 1; 
	while (d % 2 == 0): 
		d //= 2; 

	# Iterar k veces
	for i in range(k): 
		if (miillerTest(d, n) == False): 
			return False; 

	return True; 

# Código principal
# Número de iteraciones
k = 4; 

print("Todos los primos menores que 100: "); 
for n in range(1,100): 
	if (isPrime(n, k)): 
		print(n , end=" "); 

# Este código fue contribuido por mits (ver cita arriba)

#### **Paso 2: Familias universales de funciones hash**

Proporcionaremos tres funciones útiles para ti:

  - `get_random_hash_function`: Genera aleatoriamente un triple de números `(p, a, b)`, donde **p** es primo y **a** y **b** son números entre 2 y p-1. La función hash $h_{p,a,b}(n)$ se define como $ (an + b) \mod p$.
  
  - `hashfun`: Aplica la función hash aleatoria a un número `num`.
  - `hash_string`: Aplica la función hash a una cadena `hstr`. Ten en cuenta que el resultado estará entre 0 y p-1. Si tu tabla hash tiene tamaño `m`, debes tomar el resultado módulo `m` cuando llames a `hash_string`.
  
Por favor, utiliza estas funciones en tu código a continuación.

In [ ]:
# Obtener un triple aleatorio (p, a, b) donde p es primo y a, b son números entre 2 y p-1
def get_random_hash_function():
    n = random.getrandbits(64)
    if n < 0: 
        n = -n 
    if n % 2 == 0:
        n = n + 1
    while not isPrime(n, 20):
        n = n + 1
    a = random.randint(2, n-1)
    b = random.randint(2, n-1)
    return (n, a, b)

# Función hash para un número
def hashfun(hfun_rep, num):
    (p, a, b) = hfun_rep
    return (a * num + b) % p

# Función hash para una cadena.
def hash_string(hfun_rep, hstr):
    n = hash(hstr)
    return hashfun(hfun_rep, n)    

#### **Paso 3: Cargando datos**

Vamos a cargar dos archivos, `great-gatsby-fitzgerald.txt` y `war-and-peace-tolstoy.txt`, para obtener el texto de dos novelas famosas, cortesía del Proyecto Guttenberg. Filtraremos todas las palabras con longitud mayor o igual a 5 y también contaremos la frecuencia de cada palabra en un diccionario. Esto será rápido porque se utilizarán tablas hash (diccionarios) altamente optimizados incorporados en Python.

In [ ]:
filename = 'great-gatsby-fitzgerald.txt'
with open(filename, 'r', encoding='utf-8') as file:
    txt = file.read()

txt = txt.replace('\n', ' ')
words = txt.split(' ')
longer_words_gg = list(filter(lambda s: len(s) >= 5, words))
print(len(longer_words_gg))

# Contemos la frecuencia exacta de cada palabra
word_freq_gg = {}
for elt in longer_words_gg:
    if elt in word_freq_gg:
        word_freq_gg[elt] += 1
    else:
        word_freq_gg[elt] = 1
print(len(word_freq_gg))


In [ ]:
filename = 'war-and-peace-tolstoy.txt'
with open(filename, 'r', encoding='utf-8') as file:
    txt = file.read()

txt = txt.replace('\n', ' ')
words = txt.split(' ')
longer_words_wp = list(filter(lambda s: len(s) >= 5, words))
print(len(longer_words_wp))

# Contemos la frecuencia exacta de cada palabra
word_freq_wp = {}
for elt in longer_words_wp:
    if elt in word_freq_wp:
        word_freq_wp[elt] += 1
    else:
        word_freq_wp[elt] = 1
print(len(word_freq_wp))


#### **Problema 1: Implementar count-min sketch**

Implementa la clase `CountMinSketch` a continuación, donde `num_counters` es el número de contadores. Se te proporciona el constructor que ya genera un representante aleatorio de una familia de funciones hash. Implementa las funciones:
  - `increment`
  - `approximateCount`.
  
Por favor, lee cuidadosamente el constructor: este inicializa los contadores y genera la función hash para ti.
Además, cuando llames a la función `hash_string` definida anteriormente, no olvides tomar el resultado módulo m.

In [ ]:
# Clase para implementar un count-min sketch con una sola "banca" de contadores
class CountMinSketch:
    # Inicializa con `num_counters`
    def __init__(self, num_counters):
        self.m = num_counters
        self.hash_fun_rep = get_random_hash_function()
        self.counters = [0] * self.m
    
    # tu código aquí
    
    # Función: increment
    # Dada una palabra, incrementa su cuenta en el count-min sketch
    def increment(self, word):
        # tu código aquí
        
    # Función: approximateCount
    # Dada una palabra, obtiene su cuenta aproximada
    def approximateCount(self, word):
        # tu código aquí


In [ ]:
# Ahora implementaremos el algoritmo para una banca de k contadores

# Inicializa k contadores diferentes
def initialize_k_counters(k, m): 
    return [CountMinSketch(m) for i in range(k)]

# Función increment_counters
# Incrementa cada uno de los contadores individuales con la palabra
def increment_counters(count_min_sketches, word):
    # tu código aquí
    
# Función: approximate_count
# Obtiene la cuenta aproximada consultando cada banco de contadores y tomando el mínimo
def approximate_count(count_min_sketches, word):
    return min([cms.approximateCount(word) for cms in count_min_sketches])

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt 

# Veamos qué tan bien funciona tu solución para las palabras de The Great Gatsby
cms_list = initialize_k_counters(5, 1000)
for word in longer_words_gg:
    increment_counters(cms_list, word)

discrepencies = []
for word in longer_words_gg:
    l = approximate_count(cms_list, word)
    r = word_freq_gg[word]
    assert(l >= r)
    discrepencies.append(l - r)
    
plt.hist(discrepencies)

assert(max(discrepencies) <= 200), 'La mayor discrepancia debe ser definitivamente menor que 200 con alta probabilidad. Por favor revisa tu implementación'
print('Todas las pruebas superadas!')

In [ ]:
# Veamos qué tan bien funciona tu solución para War and Peace
cms_list = initialize_k_counters(5, 5000)
for word in longer_words_wp:
    increment_counters(cms_list, word)

discrepencies = []
for word in longer_words_wp:
    l = approximate_count(cms_list, word)
    r = word_freq_wp[word]
    assert(l >= r)
    discrepencies.append(l - r)

plt.hist(discrepencies)
print('Todas las pruebas superadas!')

#### **Problema 1B**

Verifica los datos obtenidos anteriormente con cálculos similares a los realizados en clase. Si tuviéramos
$5$ bancos de contadores con $5000$ contadores cada uno y una familia uniforme de funciones hash, ¿cuál es la probabilidad de que, al contar un total de $N = 2.5\times 10^{5}$ palabras, tengamos una discrepancia de 80 o más?


Tu respuesta

#### **Problema 2: Usar un filtro Bloom para contar palabras comunes**

En este problema, implementaremos un filtro Bloom para contar cuántos elementos de `longer_words_wp` (las palabras de longitud 5 o más en War and Peace) aparecen en la novela *The Great Gatsby*. Para ello, haremos lo siguiente:

 - Instanciar un filtro Bloom con un número de bits `n` y un número de funciones hash `k`.
 - Insertar todas las palabras de The Great Gatsby en el filtro.
 - Para cada palabra de War and Peace, verificar la pertenencia en el filtro Bloom y contar el número de respuestas afirmativas.

In [ ]:
class BloomFilter:
    def __init__(self, nbits, nhash):
        self.bits = [False] * nbits  # Inicializar todos los bits a False
        self.m = nbits
        self.k = nhash
        # Obtener k funciones hash aleatorias
        self.hash_fun_reps = [get_random_hash_function() for i in range(self.k)]
    
    # Función para insertar una palabra en el filtro Bloom.
    def insert(self, word):
        # tu código aquí
        
    # Verificar si una palabra pertenece al filtro Bloom
    def member(self, word):
        # tu código aquí

        return True


In [ ]:
# Realizar el conteo exacto
# Es una medida de lo optimizadas que están las estructuras de datos de Python internamente, ya que esta operación termina muy rápidamente.
all_words_gg = set(longer_words_gg)
exact_common_wc = 0
for word in longer_words_wp:
    if word in all_words_gg:
        exact_common_wc += 1
print(f'Conteo exacto de palabras comunes = {exact_common_wc}')

In [ ]:
# Intentemos usar lo mismo con un filtro Bloom.
bf = BloomFilter(100000, 5)
for word in longer_words_gg:
    bf.insert(word)
    
for word in longer_words_gg:
    assert bf.member(word), f'La palabra: {word} debería pertenecer'

common_word_count = 0
for word in longer_words_wp:
    if bf.member(word):
        common_word_count += 1
print(f'El número de palabras comunes de longitud >= 5 es: {common_word_count}')
assert common_word_count >= exact_common_wc
print('Todas las pruebas superadas: 10 puntos')

#### **Problema 2B**

Dado un filtro Bloom con $m = 100000$ bits y $k = 5$ funciones hash que asignan cada clave de forma uniforme y aleatoria a uno de los bits (según el supuesto), estima la probabilidad de que los $k$ bits $i_1, \ldots, i_k$ estén simultáneamente activados cuando se insertan $n = 10000$ palabras. Asume que la activación de cada bit es independiente de la de otro.

Tu respuesta

### **Más ejercicios**

#### **Ejercicio 1. Análisis del error en un count‑min sketch**

**Planteamiento:**  
Considera un count‑min sketch con $ k $ bancos de contadores, cada uno de tamaño $ m $. Se inserta un total de $ N $ palabras en el esquema.  
1. Demuestra que para cualquier palabra, la cuenta aproximada $ \widetilde{f} $ satisface  
   $$
   f \leq \widetilde{f} \leq f + \epsilon
   $$
   donde $ f $ es la frecuencia real y $ \epsilon $ está acotada en función de $ \frac{N}{m} $.  
2. Bajo el supuesto de funciones hash uniformemente distribuidas, usa la desigualdad de Markov para acotar la probabilidad de que la diferencia $ \widetilde{f} - f $ sea mayor o igual a un umbral dado (por ejemplo, 80).  
3. Reflexiona sobre cómo influyen los parámetros $ k $ y $ m $ en el error esperado y en la probabilidad de obtener grandes discrepancias.

#### **Ejercicio 2. Probabilidad de colisiones en count‑min sketch**

**Planteamiento:**  
Dado que cada función hash en el count‑min sketch se supone que asigna las palabras de forma uniforme entre $ m $ contadores, analiza lo siguiente:  

1. Para una palabra con frecuencia $ f $, determina la probabilidad de que, en un banco de contadores, se produzca una colisión con palabras distintas que hayan sido mapeadas al mismo contador.  
2. Utilizando la esperanza del número total de inserciones $ N $, determina la esperanza de error (incremento por colisiones) en un único contador.  
3. Discute cómo estas colisiones afectan el error agregado cuando se toman las mínimas lecturas de entre los $ k $ bancos.

#### **Ejercicio 3. Análisis del filtro Bloom**

**Planteamiento:**  
Un filtro Bloom utiliza $ m $ bits y $ k $ funciones hash para insertar $ n $ elementos.  
1. Deriva la fórmula para la probabilidad de que un bit en el filtro permanezca desactivado después de insertar $ n $ elementos.  
2. A partir de lo anterior, demuestra que la probabilidad de falso positivo (es decir, la probabilidad de que para un elemento no insertado, todos los $ k $ bits correspondientes estén activados) es:  
   $$
   P_{fp} \approx \left(1 - e^{-kn/m}\right)^k.
   $$
3. Discute cómo varía $ P_{fp} $ al modificar $ m $, $ k $ y $ n $ y encuentra (teóricamente) el número óptimo de funciones hash $ k $ para una configuración dada.

#### **Ejercicio 4. Comparación teórica: count‑min sketch vs. filtro Bloom** 
**Planteamiento:**  
Realiza un análisis comparativo en el que se discutan los siguientes aspectos:  
1. **Uso de memoria:** ¿Cómo afecta el tamaño de las estructuras (número de contadores en el count‑min sketch versus número de bits en el filtro Bloom) la eficiencia en el consumo de memoria?  
2. **Velocidad de operación:** Compara la eficiencia en la inserción y consulta para cada estructura, considerando el costo de calcular múltiples funciones hash.  
3. **Exactitud vs. aproximación:** Mientras que el count‑min sketch estima frecuencias (siempre sobreestimándolas), el filtro Bloom sólo indica pertenencia con una tasa de falsos positivos. Reflexiona sobre en qué escenarios uno podría preferir una técnica en lugar de la otra.

#### **Ejercicio 5. Propiedades de una familia de funciones hash universales** 
**Planteamiento:**  
1. Define formalmente qué significa que una familia de funciones hash sea *universal* y por qué esta propiedad es crucial para evitar malos comportamientos en estructuras como el count‑min sketch y el filtro Bloom.  
2. Demuestra que, para dos entradas distintas $ x $ e $ y $, la probabilidad de que se produzca una colisión (es decir, $ h(x) = h(y) $) es acotada por $ \frac{1}{p} $ (o $ \frac{1}{m} $ si se toma módulo $ m $) bajo el supuesto de una distribución uniforme.  
3. Explica con tus propias palabras la importancia de la "distribución uniforme" y cómo afecta la precisión de las estimaciones.

#### **Ejercicio 6. Simulación y modelado teórico**  
**Planteamiento:**  
Propón un método teórico (y, de ser posible, complementario a simulaciones experimentales) para estimar la distribución del error en las cuentas aproximadas obtenidas con un count‑min sketch.  
1. Describe cómo diseñar un experimento (ya sea mediante un análisis probabilístico o una simulación) para obtener la distribución empírica de $ \widetilde{f} - f $ para una palabra dada.  
2. Contrasta estos resultados empíricos con la cota teórica que obtuviste en el ejercicio 1.  
3. Discute posibles causas de discrepancias entre los valores teóricos y los resultados empíricos.


Tus respuestas